In [2]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from tqdm import tqdm
from copy import deepcopy
import torch
import torch.nn as nn
import torch.nn.functional as F
import rdkit
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from common.cfg_utils import get_config
from common.metrics import *
from common.losses import *
from common.cfg_utils import *
from common.old_routine import *
from datasets.make_dataset import *
from models.gnn_bind import *
from models.learnable_ff import LearnableFF
from routines.ai_routine import *
from terrace.batch import *
from terrace.type_data import *
from terrace.comp_node import *
from torchmetrics import *
from datasets.data_types import *
from models.val_model import *
from datasets.bigbind_screen import *
from datasets.pdbbind import *
from datasets.lit_pcba import *
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from common.cache import *
from glob import glob
import wandb
from git_timewarp import GitTimeWarp

In [5]:
cfg = get_config("../configs", "short_thicc_op_gnn")

In [8]:
pdb_ids = []
with open("../pdb_2016_core_set_ids.txt") as f:
    for line in f.readlines():
        pdb_id = line.split("/")[0].upper()
        pdb_ids.append(pdb_id)

dist = 5
for pdb_id in pdb_ids:
    if not os.path.exists(cfg.platform.pdbbind_dir + "/" + pdb_id):
        print(":eyes: doesn't exist")
        continue
    rec_file = cfg.platform.pdbbind_dir + f"/{pdb_id}/{pdb_id}_PRO.pdb"
    lig_file = cfg.platform.pdbbind_dir + f"/{pdb_id}/{pdb_id}_LIG.sdf"
    out_file = cfg.platform.pdbbind_dir + f"/{pdb_id}/process.pml"
    rec_name = f"{pdb_id}_PRO"
    out_pdb = "pocket.pdb"
    print(f"Writing to {out_file}")
    with open(out_file, "w") as f:
        f.write(f"load {rec_file}\n")
        f.write(f"load {lig_file}\n")
        f.write(f"select byres (({rec_name} within {dist} of organic) and not organic)\n")
        f.write(f"save {out_pdb}, sele\n")

Writing to /home/boris/Data/PDBbind_refined_2019/1BCU/process.pml
Writing to /home/boris/Data/PDBbind_refined_2019/1BZC/process.pml
Writing to /home/boris/Data/PDBbind_refined_2019/1E66/process.pml
Writing to /home/boris/Data/PDBbind_refined_2019/1EBY/process.pml
Writing to /home/boris/Data/PDBbind_refined_2019/1G2K/process.pml
Writing to /home/boris/Data/PDBbind_refined_2019/1GPK/process.pml
Writing to /home/boris/Data/PDBbind_refined_2019/1GPN/process.pml
Writing to /home/boris/Data/PDBbind_refined_2019/1H22/process.pml
Writing to /home/boris/Data/PDBbind_refined_2019/1H23/process.pml
Writing to /home/boris/Data/PDBbind_refined_2019/1K1I/process.pml
Writing to /home/boris/Data/PDBbind_refined_2019/1LPG/process.pml
Writing to /home/boris/Data/PDBbind_refined_2019/1MQ6/process.pml
Writing to /home/boris/Data/PDBbind_refined_2019/1NC1/process.pml
Writing to /home/boris/Data/PDBbind_refined_2019/1NC3/process.pml
Writing to /home/boris/Data/PDBbind_refined_2019/1NVQ/process.pml
Writing to